In [1]:
# Libraries import 
import pdbufr
import sys
import traceback
 
from math import isnan

from eccodes import *
from ecmwf.opendata import Client
from math import isnan
from datetime import datetime, timedelta

import os
import birdy
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import requests

from ipywidgets import interact
import ipyleaflet
import ipywidgets as widgets

from Magics import macro as magics
from IPython.display import display
from Magics.macro import *

import warnings
warnings.filterwarnings("ignore")

from utils import *

In [2]:
## Widget to define the starting date of the forecast of which we want to download the data ##

start_date_forecast = widgets.DatePicker(
    description = 'Forecast date:',
    value = datetime(datetime.now().year, datetime.now().month, datetime.now().day, hour=0),
)

start_date_forecast.style.description_width = '90px'

# Print widget
start_date_forecast

DatePicker(value=datetime.datetime(2023, 8, 22, 0, 0), description='Forecast date:', step=1, style=Description…

In [5]:
## Download the tracks forecast based on the starting date selected ##
# If too early in the morning the forecast of the current day cannot be donwloaded, need to fix this -> If raiseError: download forecast of the previous day
client = Client(source="azure")
client.retrieve(
    date=int(start_date_forecast.value.strftime("%Y%m%d")),
    time=0,
    stream="enfo",
    type="tf",
    step=240,
    target="data/tc_test_track_data.bufr",
);

## Donwload the observed data from IBTrACS##

url = 'https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/csv/ibtracs.ACTIVE.list.v04r00.csv'
r = requests.get(url, allow_redirects=True)
save_file = 'data/ibtracs.ACTIVE.list.v04r00.csv'
with open(save_file, 'wb') as f:
    f.write(r.content)

20230822000000-240h-enfo-tf.bufr:   0%|          | 0.00/839k [00:00<?, ?B/s]

In [8]:
## Load forecasted tracks data in a dataframe ##
# create_storms_df loads a file called tc_test_track_data.bufr
df_storms_forecast = create_storms_df()

## Load observed tracks data in a dataframe ## 
df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])

print('Observed tracks: ', df_storms_observed.NAME.squeeze().unique().tolist())
print('Forecasted tracks: ', df_storms_forecast.stormIdentifier.unique())

## Find the correspondent cyclone across forecast and observations ## 
storms_pair = storms_pair(df_storms_forecast, df_storms_observed)

Observed tracks:  ['DORA', 'LAN', 'FERNANDA', 'GREG', 'HILARY', 'NOT_NAMED']
Forecasted tracks:  ['06L' '07L' '08L' '09L']


In [9]:
# Cyclone dropdown selection
cyclone = widgets.Dropdown(
    options = storms_pair,
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'

In [10]:
cyclone

Dropdown(description='Active Storms:', options=('06L-NOT_NAMED', '06L-NOT_NAMED', '08L-NOT_NAMED', '09L-HILARY…